프로젝트 설명 및 기본 설정

In [70]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")

# 한글 폰트 설정
import matplotlib.font_manager as fm
font_path = './fonts/NanumGothic.ttf'
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'NanumGothic'

# 현재 작업 경로 확인
current_directory = os.getcwd()
print(current_directory)

c:\Users\taehyun\github-classroom\Konkuk-Univ-Glocal-Campus\ml202401-final-taehyunkim32


1. 데이터 읽고 데이터 구조 확인

In [71]:
raw_df = pd.read_csv("./amazon_uk_shoes_products_dataset_2021_12.csv")
print(f"* 총 리뷰 개수: {raw_df.shape[0]} *")
print(raw_df.info())

* 총 리뷰 개수: 6823 *
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823 entries, 0 to 6822
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                6823 non-null   object 
 1   product_name       6823 non-null   object 
 2   reviewer_name      6823 non-null   object 
 3   review_title       6822 non-null   object 
 4   review_text        6814 non-null   object 
 5   review_rating      6823 non-null   float64
 6   verified_purchase  6823 non-null   bool   
 7   review_date        6823 non-null   object 
 8   helpful_count      1953 non-null   object 
 9   uniq_id            6823 non-null   object 
 10  scraped_at         6823 non-null   object 
dtypes: bool(1), float64(1), object(9)
memory usage: 539.8+ KB
None


실제데이터를 확인

In [72]:
# 첫 번째 행의 열 이름과 값을 출력
for column in raw_df.columns:
    print(f"{column} : {raw_df.iloc[0][column]}")

url : https://www.amazon.co.uk/dp/B07SBX32T5
product_name : Klasified Women's Transparent Clear Sneaker Shoe, White, 5.5 UK
reviewer_name : Jocelyn McSayles
review_title : Love em
review_text : Love these. Was looking for converses and these were half the price and so unique— I’ve never seen clear shoes like these; they fit great. The plastic takes a little getting used to but the style is so worth it.
review_rating : 5.0
verified_purchase : True
review_date : Reviewed in the United States on 2 June 2020
helpful_count : 2 people found this helpful
uniq_id : 36eae4e5-2894-5279-a0b7-d2b330e2b814
scraped_at : 24/12/2021 02:26:25


2.데이터 전처리
1)중복데이터 제거
2)주요 특성 설정
3)결측치 제거
4)노이즈 제거
5)토큰화
6)표제어 추출

1)중복 데이터 확인 및 제거
68~~개였는데 중복된 행의 개수가 364
예시 뭐시기

In [73]:
# uniq_id 열을 기준으로 중복된 행 확인
dup = raw_df.duplicated(subset=['uniq_id'], keep=False)

# 중복된 행의 개수
count_dup = dup.sum()
print(f"중복된 행의 개수: {count_dup}")

# 중복된 uniq_id 예시 추출
dup_uniq_id = raw_df['uniq_id'][raw_df['uniq_id'].duplicated()].iloc[0]

# 중복된 uniq_id 출력
print(f"중복된 uniq_id: {dup_uniq_id}")

# 중복 리뷰 케이스 확인
selected_columns = ['review_title', 'review_text', 'review_rating', 'uniq_id']
filtered_rows = raw_df[raw_df['uniq_id'] == dup_uniq_id][selected_columns]

# 결과 출력
filtered_rows

중복된 행의 개수: 364
중복된 uniq_id: d1871977-2e70-5d3d-bcb9-888819c4d0d6


,review_title,review_text,review_rating,uniq_id
1002,Per ben due volte taglia errata,Per ben due volte ho dovuto restituire l'artic...,1.0,d1871977-2e70-5d3d-bcb9-888819c4d0d6
1828,Per ben due volte taglia errata,Per ben due volte ho dovuto restituire l'artic...,1.0,d1871977-2e70-5d3d-bcb9-888819c4d0d6


원본 데이터: 6823
중복 데이터: 364
중복 데이터를 제외한 데이터: 6823 - (364 / 2) = 6641

In [74]:
# uniq_id 열을 기준으로 중복된 행 제거
df_unique = raw_df.drop_duplicates(subset=['uniq_id'], keep='first')

# 중복이 제거된 총 리뷰의 개수
count_unique = len(df_unique)
print(f"중복 데이터를 제외한 행의 개수: {count_unique}")

중복 데이터를 제외한 행의 개수: 6641


2)주요 특성 설정
`review_title`, `review_text`, `review_rating` 3가지 칼럼을 주요 특성으로 사용

In [75]:
# 사용할 columns 선택
df_selected = df_unique[["review_title", "review_text", "review_rating"]]
df_selected.head(5)

,review_title,review_text,review_rating
0,Love em,Love these. Was looking for converses and thes...,5.0
1,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0
2,Good quality,Good quality,5.0
3,Good,Great,5.0
4,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0


2)결측치 확인 및 제거

In [76]:
print(f"제목 결측치 개수: {df_selected['review_title'].isnull().sum()}")
print(f"리뷰 결측치 개수: {df_selected['review_text'].isnull().sum()}")
print(f"레이팅 결측치 개수: {df_selected['review_rating'].isnull().sum()}")

제목 결측치 개수: 1
리뷰 결측치 개수: 8
레이팅 결측치 개수: 0


제목, 리뷰에 결측치가 존재하는 행은 제거
중복 데이터를 제외한 데이터: 6823
결측치가 존재하는 데이터 : 1 + 8 = 9
결측치를 제외한 데이터: 6823 - (364 / 2) = 6641

In [78]:
df_cleaned = df_selected.dropna(subset=["review_text", "review_title"])

# 결측치가 제거된 총 리뷰의 개수
count_cleand = len(df_cleaned)
print(f"결측치를 제외한 행의 개수: {count_cleand}")

결측치를 제외한 행의 개수: 6632


,review_title,review_text,review_rating
0,Love em,Love these. Was looking for converses and thes...,5.0
1,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0
2,Good quality,Good quality,5.0
3,Good,Great,5.0
4,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0


In [82]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# 필요한 NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 전처리 함수 선언
def data_preprocessing(contents):
    # 노이즈 제거 (소문자로 변환, 숫자 제거, 다중 공백 제거, 특수 문자 제거)
    contents = re.sub(r'[^\w\s]', '', re.sub(r'\s+', ' ', re.sub(r'\d+', '', contents.lower())))

    # 토큰화
    tokens = word_tokenize(contents)
    
    # 불용어 제거
    stopword = [word for word in tokens if word not in stopwords.words("english")]
    
    # 표제어 추출
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in stopword]
    
    return ' '.join(tokens)

# 텍스트 전처리
df_cleaned["review_text"] = df_cleaned["review_text"].apply(data_preprocessing)
df_cleaned["review_title"] = df_cleaned["review_title"].apply(data_preprocessing)

# 전처리 완료
df = df_cleaned

df.head(10)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\taehyun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taehyun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\taehyun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\taehyun\AppData\Roaming\nltk_data...


,review_title,review_text,review_rating
0,love em,love looking converse half price unique ive ne...,5.0
1,plastic ripped,shoe cute nd day wearing tongue started rippin...,2.0
2,good quality,good quality,5.0
3,good,great,5.0
4,perfette,ho scelto il modello bianco con rifinitura die...,5.0
5,delusione,sono solita comprare scarpe guess e non ho mai...,1.0
6,molto belle,le scarpe sono molto belle calzano perfettamente,5.0
7,molto carine e comode,semplicemente perfette io uso plantari su misu...,5.0
8,bellissimepeccato per il numero,le scarpe sono bellissime arrivate perfetto st...,5.0
9,scarpe ben realizzate venditore ottimo,le scarpe sono ben fatte e il venditore molto ...,5.0
